#### Prepare packages

In [ ]:
!pip install ase
!git clone https://github.com/yilinyang1/GPR-kernel.git

     |████████████████████████████████| 2.2MB 6.0MB/s 
Cloning into 'GPR-kernel'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 22 (delta 2), reused 22 (delta 2), pack-reused 0
Unpacking objects: 100% (22/22), done.


In [ ]:
from ase.db import connect
import numpy as np
from scipy.linalg import solve_triangular
from numpy.linalg import cholesky, det
from scipy.optimize import minimize

#### Prepare data

For each config with N atoms, x is a vector of 3N (cartesian coordinates), y is a vector of 1 + 3N dimensions with energy and forces.  
For a training set with M configurations, X is a matrix of [M, 3N], y is a vector of [M * (1+3N)]

In [ ]:
def atoms2data(images):
    X = []
    y = []
    for atoms in images:
        entry_X = atoms.positions.flatten()
        entry_y_nrg = atoms.get_potential_energy()
        # negative because in kernel we deal with positive derivative
        entry_y_frs = -1.0 * atoms.get_forces().flatten()
        entry_y = np.concatenate([[entry_y_nrg], entry_y_frs], axis=0)
        X.append(entry_X)
        y.append(entry_y)
    
    y_new = []  # reorder y
    n, d = len(images), 1 + len(images[0]) * 3
    for i in range(d):
        for j in range(n):
            y_new.append(y[j][i])
    return np.array(X), np.array(y_new)

In [ ]:
db = connect('./GPR-kernel/acrolein-AgPd-sample.db')
images = [entry.toatoms() for entry in db.select()]

n_atoms = len(images[0])
X_train, y_train = atoms2data(images[0:2])  # first two images as training set
X_valid, _ = atoms2data([images[2], images[3]])  # third and forth images as test set

#### Build GP potential
 - prior: max energies in training set and zero forces according to the paper
 - kernel between two data points
 - kernel between one test data with the training set (this is enough for this application since during relaxation, we need to predict energy and forces of configurations sequentially)
 - calculate likelihood of the training set
 - optimization of the hyperparameters based on likelihood

##### Hyperparameters: (based on https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.122.156001)
 - signal noise before kernel, $\sigma_f$, set as 1.0
 - energy data noise: $\sigma_n^e$, optimize between [0.001, 0.01]
 - force data noise: $\sigma_n^f$, optiomize between [0.0001, 0.001]
 - length scale: $l_m$, optimize between [0.01, $d_{path}$], $d_{path}$ is the Euclidean distance of the last NEB path (not sure the meaning of it), set as 1.0

In [ ]:
# three types of kernel: energy-energy, energy-force, force-force
def ee_kernel(x, xp, lens):
    """
    x: (d, ), positions, row
    xp: (d, ), positions, col
    lens: (d, ), length scales
    """
    inner = 0.5 * (np.square((x - xp) / lens)).sum()
    return 1.0 * np.exp(-inner)

def ef_kernel(x, xp, lens, d):
    """
    x: (d, ), positions, row
    xp: (d, ), positions, col
    lens: (d, ), length scales
    d: which dimension of force or cartisian coordinate
    """
    pre = 1.0 * ((x[d] - xp[d])) / (lens**2)
    inner = 0.5 * (np.square((x - xp) / lens)).sum()
    return pre * np.exp(-inner)

def fe_kernel(x, xp, lens, d):
    """
    x: (d, ), positions, row
    xp: (d, ), positions, col
    lens: (d, ), length scales
    d: which dimension of force or cartisian coordinate
    """
    pre = -1.0 * ((x[d] - xp[d])) / (lens**2)
    inner = 0.5 * (np.square((x - xp) / lens)).sum()
    return pre * np.exp(-inner)

def ff_kernel(x, xp, lens, d, dp):
    """
    x: (d, ), positions, row
    xp: (d, ), positions, col
    lens: (d, ), length scales
    d: which dimension of force or cartisian coordinate
    dp: which dimension of force or cartisian coordinate
    """
    delta = 1 if d == dp else 0
    pre = 1.0 / (lens ** 2) * (delta - (x[d] - xp[d]) * (x[dp] - xp[dp]) / (lens ** 2))
    inner = 0.5 * (np.square((x - xp) / lens)).sum()
    return pre * np.exp(-inner)

def kernel_train(X_train, lens):
    """
    X: [m1, d], positions
    lens: length scales
    return: kernel matrix for the training data
    [[K,     Kgd], 
     [Kgd^T, Kdd]]
    K: [n, n]  for energy
    Kgd: [n, nd]  for energy and forces
    Kdd: [nd, nd] for forces and forces
    
    The forces are ordered as: [f_11, f_21, ...f_n1, ..., f_1d, f_2d, ..., fnd]
    """
    n, dim = len(X_train), len(X_train[0])
    res = np.zeros([n * (1 + dim), n * (1 + dim)])
    
    # fill the energy kernel matrix K
    for i in range(n):
        for j in range(n):
            if i <= j:
                res[i][j] = ee_kernel(X_train[i], X_train[j], lens)
            else:
                res[i][j] = res[j][i]
    
    # fill the energy and force kernel matrix
    for i in range(n):
        for j in range(n, n * (1 + dim)):
            config_id = j % n
            d = j // n - 1
            res[i][j] = ef_kernel(X_train[i], X_train[config_id], lens, d)
    
    # fill the force and energy kernel matrix
    for i in range(n, n * (1 + dim)):
        for j in range(n):
            res[i][j] = res[j][i]
            
    # fill the force and force kernel matrix
    for i in range(n, n * (1 + dim)):
        for j in range(n, n * (1 + dim)):
            if i <= j:
                id1, d1 = i % n, i // n - 1
                id2, d2 = j % n, j // n - 1
                value = ff_kernel(X_train[id1], X_train[id2], lens, d1, d2)
                res[i][j] = value
            else:
                res[i][j] = res[j][i]
    
    return res

def nll_obs(K, y, sigma_e, sigma_f, n):
    """
    K: kernel matrix of the training set
    y: observation of the training set
    sigma_e: noise of energy
    sigma_f: noise of forces
    n: number of training instances
    """
    diag = np.zeros_like(K)
    for i in range(n):
        diag[i][i] = sigma_e ** 2
    for i in range(n, len(diag)):
        diag[i][i] = sigma_f ** 2
    
    noise_K = K + diag
    
    # stable
    L = cholesky(noise_K)
    S1 = solve_triangular(L, y, lower=True)
    S2 = solve_triangular(L.T, S1, lower=False)
    nll = np.sum(np.log(np.diagonal(L))) + 0.5 * y.dot(S2) + 0.5 * n * np.log(2*np.pi)
    return nll


def train(X_train, y_train, l_max):
    """
    X_train: [n, 3 * d]
    y_train: [n * (1 + 3d)]
    """    
    def obj_func(theta):
        sigma_e, sigma_f, lens = theta[0], theta[1], theta[2]
        kernel_matrix = kernel_train(X_train, lens)
        return nll_obs(kernel_matrix, y_train, sigma_e, sigma_f, len(X_train))

    init_sigma_e, init_sigma_f, init_lens = 0.005, 0.0005, 1.0
    init_theta = [init_sigma_e, init_sigma_f, init_lens] 

    bnds = [[0.001, 0.01], [0.0001, 0.001], [0.01, l_max]]
    res = minimize(obj_func, init_theta, bounds=bnds)
    return res


def predict(X_train, y_train, x_test, theta):
    sigma_e, sigma_f, lens = theta[0], theta[1], theta[2]
    n_train, dim = len(X_train), X_train.shape[1]
    kernel_matrix = kernel_train(X_train, lens)
    diag = np.zeros_like(kernel_matrix)
    for i in range(n_train):
        diag[i][i] = sigma_e ** 2
    for i in range(n_train, len(diag)):
        diag[i][i] = sigma_f ** 2
    noise_kernel = kernel_matrix + diag
    
    K_test_test = kernel_train(x_test.reshape(1, -1), lens)  # [1+d, 1+d]
    
    # predict energy
    K_eX = np.zeros(len(kernel_matrix[0]))
    # fill energy-energy
    for i in range(n_train):
        K_eX[i] = ee_kernel(x_test, X_train[i], lens)
    # fill energy-force
    for i in range(n_train, n_train * (1 + dim)):
        train_id = i % n_train
        d = i // n_train - 1
        K_eX[i] = ef_kernel(x_test, X_train[train_id], lens, d)
    
    
    mu_nrg = K_eX @ np.linalg.inv(noise_kernel) @ y_train
    var_nrg = K_test_test[0][0] - K_eX @ np.linalg.inv(noise_kernel) @ K_eX.T
    
    # predict forces
    K_fX = np.zeros([dim, len(kernel_matrix[0])])
    # fill force-energy
    for i in range(dim):
        for j in range(n_train):
            K_fX[i][j] = fe_kernel(x_test, X_train[j], lens, i)
    # fill force-force
    for i in range(dim):
        for j in range(n_train, n_train * (1 + dim)):
            train_id = j % n_train
            train_d = j // n_train - 1
            K_fX[i][j] = ff_kernel(x_test, X_train[train_id], lens, i, train_d)
    
    mu_frs = K_fX @ np.linalg.inv(noise_kernel) @ y_train
    var_frs = np.diag(K_test_test)[1:] - np.diag(K_fX @ np.linalg.inv(noise_kernel) @ K_fX.T)
    
    return mu_nrg, var_nrg, -1.0 * mu_frs.reshape(-1, 3), var_frs.reshape(-1, 3)

In [ ]:
def evaluate(nrg_pred, frs_pred, nrg_label, frs_label):
    nrg_err = abs(nrg_pred - nrg_label)
    frs_err = abs(frs_pred - frs_label)
    return nrg_err, frs_err

#### Training

In [ ]:
%%time
opt_res = train(X_train, y_train, 1.0)

CPU times: user 9.64 s, sys: 5.05 s, total: 14.7 s
Wall time: 9.14 s


#### Evaluation

In [ ]:
# first test sample
x_test = X_valid[0]
mu_nrg, var_nrg, mu_frs, var_frs = predict(X_train, y_train, x_test, opt_res.x)
nrg_label = images[2].get_potential_energy()
frs_label = images[2].get_forces()
nrg_err, frs_err = evaluate(mu_nrg, mu_frs, nrg_label, frs_label)
print(f'nrg mae: {nrg_err:.3f} eV, nrg std: {var_nrg**0.5:.3f} force mae: {frs_err.mean():.3f} eV/AA force mean std: {(var_frs**0.5).mean():.3f} eV/AA')

nrg mae: 0.019 eV, nrg std: 0.002 force mae: 0.068 eV/AA force mean std: 0.048 eV/AA


In [ ]:
# second test sample
x_test = X_valid[1]
mu_nrg, var_nrg, mu_frs, var_frs = predict(X_train, y_train, x_test, opt_res.x)
nrg_label = images[3].get_potential_energy()
frs_label = images[3].get_forces()
nrg_err, frs_err = evaluate(mu_nrg, mu_frs, nrg_label, frs_label)
print(f'nrg mae: {nrg_err:.3f} eV, nrg std: {var_nrg**0.5:.3f} force mae: {frs_err.mean():.3f} eV/AA force mean std: {(var_frs**0.5).mean():.3f} eV/AA')

nrg mae: 0.064 eV, nrg std: 0.003 force mae: 0.194 eV/AA force mean std: 0.073 eV/AA


#### C++ implementation

When the size of the training set grows up, the training time increases (scales cubic with the number of training points). Use C++ to accelerate this process.

In [ ]:
# build the C++ module
! cd GPR-kernel/cpp_utils/ && python libkernel_builder.py
! cp -r GPR-kernel/cpp_utils ./cpp_utils

generating ./_libkernel.cpp
the current directory is '/content/GPR-kernel/cpp_utils'
running build_ext
building '_libkernel' extension
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-a56wZI/python3.7-3.7.10=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-a56wZI/python3.7-3.7.10=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I./ -I/usr/include/python3.7m -c _libkernel.cpp -o ./_libkernel.o
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-a56wZI/python3.7-3.7.10=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-a56wZI/python3.7-3.7.10=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I./ -I/usr/include/python3.7m -c kernels.

In [ ]:
from cpp_utils.gpr_utils_cpp import gpr_train, gpr_predict

# more training data
X_train, y_train = atoms2data(images[0:15])  # use 10 images as training set
X_valid, _ = atoms2data([images[15], images[16]])  # two test images

In [ ]:
%%time
# python version
opt_res_python = train(X_train, y_train, 1.0)

CPU times: user 3min 6s, sys: 2.78 s, total: 3min 9s
Wall time: 3min 6s


In [ ]:
%%time
# C++ version
opt_res_cpp = gpr_train(X_train, y_train, 1.0)

CPU times: user 6.32 s, sys: 2.66 s, total: 8.98 s
Wall time: 5.72 s


In [ ]:
opt_res_python

      fun: 14276.313689260465
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([-5.92626748e-01, -8.68507027e+05, -8.94860732e+03])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 24
      nit: 5
   status: 0
  success: True
        x: array([0.0010965, 0.001    , 1.       ])

In [ ]:
opt_res_cpp

      fun: 14276.313689682744
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 1.64654921e+00, -8.68500786e+05, -8.94872901e+03])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 24
      nit: 5
   status: 0
  success: True
        x: array([0.00109614, 0.001     , 1.        ])